# Exemple de Rump

In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 1 - M. Massot 2020-2021 - Ecole polytechnique
#    ----------   
#    Evaluation de la fonction de Rump 
#    
#    Auteurs : L. Séries et M. Massot - (C) 2021
#    

## Des résultats surprenants...

On considère la fonction :

$$ f(x,y) = (333.75 - x^2) y^6 + x^2 (11 x^2 y^2 - 121y^4 - 2) + 5.5 y^8 + \frac{x}{2y} $$

In [ ]:
def f_rump(x,y):
    return (333.75-x*x)*y**6 + x**2*(11*(x**2)*(y**2) - 121*y**4 - 2) + 5.5*y**8 + x/(2*y)

L'évaluation de la fonction $f$ au point $(x,y) = (77617, 33096)$ pour différents formats de nombre à virgule flottante donne les résultats suivants :

In [ ]:
from mpmath import mp

# flottants sur 32 bits 
mp.prec = 24
x = mp.mpf('77617')
y = mp.mpf('33096')
print("Evaluation de la fonction pour des flottants sur 32 bits")
print(f"f(x,y) = {f_rump(x,y)}")

# flottants sur 64 bits 
mp.prec = 53
x = mp.mpf('77617')
y = mp.mpf('33096')
print("\nEvaluation de la fonction pour des flottants sur 64 bits")
print(f"f(x,y) = {f_rump(x,y)}")

# flottants sur 128 bits 
mp.prec = 113
x = mp.mpf('77617')
y = mp.mpf('33096')
print("\nEvaluation de la fonction pour des flottants sur 128 bits")
print(f"f(x,y) = {f_rump(x,y)}")

Ces résultats semblent cohérents: on trouve pour chaque format de nombre à virgule flottante le même résultat avec une précision croissante avec la taille de la mantisse.

En réalité, la valeur correcte de l'évaluation de la fonction $f$ au point $(x,y) = (77617, 33096)$ est bien différente, elle peut s'obtenir pour une taille de maintisse au moins égale à 122 :

In [ ]:
# flottants dont la mantisse est égale à 122
mp.prec = 122
x = mp.mpf('77617')
y = mp.mpf('33096')
print(f"\nEvaluation de la fonction pour des flottants dont la mantisse est égale à {mp.prec}")
print(f"f(x,y) avec {mp.dps} chiffres significatifs = {f_rump(x,y)}")

Si on considère les formes suivantes de la même fonction :

* $ f(x,y) = 333.75 y^6 + x^2 (11 x^2 y^2 - y^6 - 121y^4 - 2) + 5.5 y^8 + \displaystyle \frac{x}{2y} $

* $ f(x,y) = 333.75 yyyyyy + xx (11 xx \; yy - yyyyyy - 121yyyy - 2) + 5.5 yyyyyyyy + \displaystyle \frac{x}{2y} $

In [ ]:
def f_rump_02(x,y):
    return 333.75*y**6 + (x**2)*(11*(x**2)*(y**2) - y**6 - 121*y**4 - 2) + 5.5*y**8 + x/(2*y)

def f_rump_03(x,y):
    return 333.75*y*y*y*y*y*y + (x*x)*(11*(x*x)*(y*y) - y*y*y*y*y*y - 121*y*y*y*y - 2) + 5.5*y*y*y*y*y*y*y*y + x/(2*y)

Les évaluations de $f_2$ et $f_3$ au point $(x,y) = (77617, 33096)$ donnent les résultats suivants :

In [ ]:
# flottants sur 32 bits 
mp.prec = 24
x = mp.mpf('77617')
y = mp.mpf('33096')
print("Evaluation de la fonction pour des flottants sur 32 bits")
print(f"f2(x,y) = {f_rump_02(x,y)}")
print(f"f3(x,y) = {f_rump_03(x,y)}")

# flottants sur 64 bits 
mp.prec = 53
x = mp.mpf('77617')
y = mp.mpf('33096')
print("\nEvaluation de la fonction pour des flottants sur 64 bits")
print(f"f2(x,y) = {f_rump_02(x,y)}")
print(f"f3(x,y) = {f_rump_03(x,y)}")

# flottants sur 128 bits 
mp.prec = 113
x = mp.mpf('77617')
y = mp.mpf('33096')
print("\nEvaluation de la fonction pour des flottants sur 128 bits")
print(f"f2(x,y) = {f_rump_02(x,y)}")
print(f"f3(x,y) = {f_rump_03(x,y)}")

# flottants dont la mantisse est égale à 122
mp.prec = 122
x = mp.mpf('77617')
y = mp.mpf('33096')
print(f"\nEvaluation de la fonction pour des flottants dont la mantisse est égale à {mp.prec}")
print(f"f2(x,y) = {f_rump_02(x,y)}")
print(f"f3(x,y) = {f_rump_03(x,y)}")
print(f"\nNombre de chiffres significatifs avec une mantisse à 122 :{mp.dps}")

Ces résultats appellent plusieurs remarques : 

* Le choix de la méthode de calcul de la fonction a un impact important sur le resultat, qui peut être radicalement différent ou similaire pour un calcul en quadruple précision mais faux !
* Clairement, quand la précision est suffisante pour un calcul exact, les deux méthodes donnent le même résultat.
* La question se pose alors de connaître l'origine des erreurs obtenues avec simple, double et quadruple précision.
* Le lecteur intéressé pourra aussi observer qu'avec Sagemath, un calcul exact dans le corps des rationnels permet d'exprimer le résultat final sous une forme d'une fraction : $-54767/66192$ et de vérifier que la dernière évaluation est correcte (voir page suivante).



## ... et des éléments d'explication

Les termes $a = 333.75 y^6 + x^2 (11 x^2 y^2 - y^6 - 121y^4 - 2)$ et $b = 5.5 y^8$ nécessitent une précision importante pour être représentés exactement :

In [ ]:
mp.prec = 121
x = mp.mpf('77617')
y = mp.mpf('33096')
a = (333.75-x*x)*y**6 + x**2*(11*(x**2)*(y**2) - 121*y**4 - 2) 
b = 5.5*y**8
print(f"Avec une mantisse de :{mp.prec}")
print()
print(f"a = {a}")
print(f"a = {mp.nstr(a, 37)}")
print(f"b = {b}")
print(f"a = {mp.nstr(b, 37)}")
print(f"a+b = {a+b}")
print()

mp.prec = 122
x = mp.mpf('77617')
y = mp.mpf('33096')
a = (333.75-x*x)*y**6 + x**2*(11*(x**2)*(y**2) - 121*y**4 - 2) 
b = 5.5*y**8
print(f"Avec une mantisse de :{mp.prec}")
print(f"a = {a}")
print(f"a = {mp.nstr(a, 37)}")
print(f"b = {b}")
print(f"a = {mp.nstr(b, 37)}")
print(f"a+b = {a+b}")

En fait, on ajoute des nombre décimaux très grands avec 36 chiffres significatifs qu'il faut absolument évaluer pour avoir une somme de $a$ et de $b$ qui fasse exactement $-2$. Avec un chiffre significatif de moins, le résultat est $0$ comme le montre l'évaluation précédente. 

La question qui se pose alors est de savoir si le problème vient de la fonction à évaluer ou de la méthode de calcul de la fonction. Un exercice dans la petite classe permettra de montrer que l'évaluation de la fonction en ce point est extrêmement mal conditionnée et que pour avoir une erreur forward non catastrophique devant le résultat, il faut une précision machine très petite et donc un nombre élevé de chiffres significatifs.